This notebook explains the use of formula langauge and capability of Statsample-GLM to handle category data in regression.


This notebook based [this](https://www.google.com/url?q=https%3A%2F%2Fnbviewer.jupyter.org%2Fgithub%2Fagisga%2Fsciruby-notebooks%2Fblob%2Fmaster%2FData%2520Analysis%2FLogistic%2520regression%2520with%2520categorical%2520data.ipynb&sa=D&sntz=1&usg=AFQjCNE7gDkrVcPcy6d4EeqtRixVhB017A) notebook created by [Alexej](http://github.com/agisga)

# Logistic regression with categorical data

We aim to fit a logistic regression model to the [shelter animal data](https://www.kaggle.com/c/shelter-animal-outcomes) from [kaggle](https://www.kaggle.com/competitions) using the Ruby gems `daru` and `statsample-glm`.



Let's first load the data.

In [1]:
require 'daru'
shelter_data = Daru::DataFrame.from_csv 'data/animal_shelter_train.csv'
p shelter_data.shape
shelter_data.head(3)

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"https://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"https://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    w

[26711, 10]


#<Daru::DataFrame(3x10)>
              AnimalID       Name   DateTime OutcomeTyp OutcomeSub AnimalType SexuponOut      Breed      Color AgeuponOut
          0    A671945    Hambone 2014-02-12 Return_to_        nil        Dog Neutered M Shetland S Brown/Whit       52.0
          1    A656520      Emily 2013-10-13 Euthanasia  Suffering        Cat Spayed Fem Domestic S Cream Tabb       52.0
          2    A686464     Pearce 2015-01-31   Adoption     Foster        Dog Neutered M Pit Bull M Blue/White      104.0

We need to tell Daru what vectors are category. We can do with via [#to_category](http://www.rubydoc.info/github/v0dro/daru/master/Daru/DataFrame#to_category-instance_method)

In [2]:
shelter_data.to_category 'OutcomeType', 'OutcomeSubtype', 'AnimalType', 'SexuponOutcome', 'Breed', 'Color'
nil

We create a 0-1-valued indicator for whether the animal got adopted. We will then create a logistic model to predict whether an animal got adopted or not.

In [3]:
shelter_data['OutcomeType_Adoption'] = (shelter_data['OutcomeType'].contrast_code)['OutcomeType_Adoption']
shelter_data.head 3

#<Daru::DataFrame(3x11)>
              AnimalID       Name   DateTime OutcomeTyp OutcomeSub AnimalType SexuponOut      Breed      Color AgeuponOut OutcomeTyp
          0    A671945    Hambone 2014-02-12 Return_to_        nil        Dog Neutered M Shetland S Brown/Whit       52.0          0
          1    A656520      Emily 2013-10-13 Euthanasia  Suffering        Cat Spayed Fem Domestic S Cream Tabb       52.0          0
          2    A686464     Pearce 2015-01-31   Adoption     Foster        Dog Neutered M Pit Bull M Blue/White      104.0          1

Before we create a model. Let's do some preprocessing to create an effective model.

## Some data preprocessing

I am using only 600 rows for this Demo because Statsample-GLM is a bit slow in computing.

In [4]:
small = shelter_data.head 600
small.head 3

#<Daru::DataFrame(3x11)>
              AnimalID       Name   DateTime OutcomeTyp OutcomeSub AnimalType SexuponOut      Breed      Color AgeuponOut OutcomeTyp
          0    A671945    Hambone 2014-02-12 Return_to_        nil        Dog Neutered M Shetland S Brown/Whit       52.0          0
          1    A656520      Emily 2013-10-13 Euthanasia  Suffering        Cat Spayed Fem Domestic S Cream Tabb       52.0          0
          2    A686464     Pearce 2015-01-31   Adoption     Foster        Dog Neutered M Pit Bull M Blue/White      104.0          1

In [5]:
p small['Breed'].categories.size, small['Color'].categories.size


1380
366


[1380, 366]

Since, the number of categories in 'Breed' and 'Color' is large, we need club some of these categories.

### Grouping Breeds

Lets have a look at the distribution.

In [6]:
small['Breed'].frequencies.sort(ascending: false).head(10)

#<Daru::Vector(10)>
                                     Breed
 Domestic Shorthair M                  204
 Chihuahua Shorthair                    47
         Pit Bull Mix                   38
 Labrador Retriever M                   33
 Domestic Medium Hair                   17
          Siamese Mix                   11
 Domestic Longhair Mi                   11
  German Shepherd Mix                   10
 Australian Cattle Do                    8
        Dachshund Mix                    7

Lets merge the infrequent occuring categories into single categories 'other' so we can have less number of categories to deal with.

Here we've used [#rename_categories](http://www.rubydoc.info/github/v0dro/daru/master/Daru/Category#rename_categories-instance_method) which accepts a hash mapping old categories to new one.

In [7]:
other_cats = small['Breed'].categories.select { |i| small['Breed'].count(i) < 10 }
other_cats_hash = other_cats.zip(['other']*other_cats.size).to_h
small['Breed'].rename_categories other_cats_hash
small['Breed'].frequencies

#<Daru::Vector(9)>
                                     Breed
 Domestic Shorthair M                  204
         Pit Bull Mix                   38
  German Shepherd Mix                   10
 Chihuahua Shorthair                    47
 Labrador Retriever M                   33
 Domestic Longhair Mi                   11
          Siamese Mix                   11
 Domestic Medium Hair                   17
                other                  229

And let's set the base category to 'other'.

In [8]:
small['Breed'].base_category = 'other'

"other"

We now do the same with 'Colors'

### Grouping colors

In [9]:
p small['Color'].categories.size
small['Color'].frequencies.sort(ascending: false).head 10

366


#<Daru::Vector(10)>
                               Color
       Black/White                66
             Black                52
       Brown Tabby                37
          Tricolor                22
       Brown/White                21
 Brown Tabby/White                20
            Calico                19
             White                19
         Tan/White                18
             Brown                16

In [10]:
other_cats = small['Color'].categories.select { |i| small['Color'].count(i) < 10 }
other_cats_hash = other_cats.zip(['other']*other_cats.size).to_h
small['Color'].rename_categories other_cats_hash
small['Color'].frequencies

#<Daru::Vector(24)>
                                   Color
         Brown/White                  21
          Blue/White                  12
                 Tan                  11
           Black/Tan                  14
          Blue Tabby                  10
         Brown Tabby                  37
               White                  19
               Black                  52
               Brown                  16
  Orange Tabby/White                  14
         Black/White                  66
 Brown Brindle/White                  10
        Orange Tabby                  15
     Chocolate/White                  11
                Blue                  10
                 ...                 ...

In [11]:
small['Color'].base_category = 'other'

"other"

### Looking at SexuponOutcome

In [12]:
small['SexuponOutcome'].frequencies

#<Daru::Vector(6)>
                SexuponOutcome
  Neutered Male            216
  Spayed Female            205
    Intact Male             78
  Intact Female             77
        Unknown             24
                             0

The last row tells us that there is a entry with category as 'nil'. Lets rename this category to 'Unknown' because 'Unknown' stores all the unkown values.

In [13]:
p small['SexuponOutcome'].categories
small['SexuponOutcome'].rename_categories nil => 'Unknown'
small['SexuponOutcome'].categories

["Neutered Male", "Spayed Female", "Intact Male", "Intact Female", "Unknown", nil]


["Neutered Male", "Spayed Female", "Intact Male", "Intact Female", "Unknown"]

### Split to train and test

In [14]:
train = small.head 500
test = small.tail 100
p train.size, test.size

500
100


[500, 100]

## Model fit

Now, having put data in appropriate form, we can fit the logistic regression model with `statsample-glm`.

In [16]:
m = test['OutcomeType_Adoption'].mean
"Trivial accuracy = #{[m, 1-m].max}"

"Trivial accuracy = 0.5900000000000001"

In [17]:
require 'statsample-glm'

formula = 'OutcomeType_Adoption~AnimalType+Breed+AgeuponOutcome(Weeks)+Color+SexuponOutcome'
glm_adoption = Statsample::GLM::Regression.new formula, train, :logistic
glm_adoption.df_for_regression.head 5
glm_adoption.model.coefficients :hash

{:AnimalType_Cat=>0.8376443692275163, :"Breed_Pit Bull Mix"=>0.28200753488859803, :"Breed_German Shepherd Mix"=>1.0518504638731023, :"Breed_Chihuahua Shorthair Mix"=>1.1960242033878856, :"Breed_Labrador Retriever Mix"=>0.445803000000512, :"Breed_Domestic Longhair Mix"=>1.898703165797653, :"Breed_Siamese Mix"=>1.5248210169271197, :"Breed_Domestic Medium Hair Mix"=>-0.19504965010288533, :Breed_other=>0.7895601504638325, :"Color_Blue/White"=>0.3748263925801828, :Color_Tan=>0.11356334165122918, :"Color_Black/Tan"=>-2.6507089126322114, :"Color_Blue Tabby"=>0.5234717706465536, :"Color_Brown Tabby"=>0.9046099720184905, :Color_White=>0.07739310267363662, :Color_Black=>0.859906249787038, :Color_Brown=>-0.003740755055106689, :"Color_Orange Tabby/White"=>0.2336674067343927, :"Color_Black/White"=>0.22564205490196415, :"Color_Brown Brindle/White"=>-0.6744314269278774, :"Color_Orange Tabby"=>2.063785952843677, :"Color_Chocolate/White"=>0.6417921901449108, :Color_Blue=>-2.1969040091451704, :Color_Cal

We can also predict using the model we just created.

In [18]:
predict = glm_adoption.predict test
predict.map! { |i| i < 0.5 ? 0 : 1 }
predict.head 5

#<Daru::Vector(5)>
   0   0
   1   0
   2   1
   3   0
   4   0